In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds
import os
import re
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
data_path = os.path.join(os.getenv('HOME'), 'aiffel/transformer_chatbo/data/ChatbotData .csv')

In [3]:
!pip install sentencepiece
!pip install wget

In [4]:
import math
from IPython.display import display
from tqdm import tqdm, tqdm_notebook, trange
import sentencepiece as spm
import wget

In [5]:
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split

In [6]:
data = pd.read_csv(data_path)
data

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0
...,...,...,...
11818,훔쳐보는 것도 눈치 보임.,티가 나니까 눈치가 보이는 거죠!,2
11819,훔쳐보는 것도 눈치 보임.,훔쳐보는 거 티나나봐요.,2
11820,흑기사 해주는 짝남.,설렜겠어요.,2
11821,힘든 연애 좋은 연애라는게 무슨 차이일까?,잘 헤어질 수 있는 사이 여부인 거 같아요.,2


In [7]:
data['Q+A'] = data['Q']+' '+data['A']

In [8]:
train_data_path = os.path.join(os.getenv('HOME'), 'aiffel/transformer_chatbo/data/train_data.txt')
spm_model_prefix = os.path.join(os.getenv('HOME'), 'aiffel/transformer_chatbo/data/spm_train_data')

In [9]:
all_texts = data['Q+A'].tolist()
with open('train_data.txt', 'w', encoding='utf-8') as f:  
    f.write('\n'.join(all_texts)) 

In [10]:
vocab_size = 8192
spm.SentencePieceTrainer.Train(f'--input={train_data_path} --model_prefix={spm_model_prefix} --vocab_size={vocab_size} --model_type=bpe --max_sentence_length=1024')

sentencepiece_trainer.cc(177) LOG(INFO) Running command: --input=/aiffel/aiffel/transformer_chatbo/data/train_data.txt --model_prefix=/aiffel/aiffel/transformer_chatbo/data/spm_train_data --vocab_size=8192 --model_type=bpe --max_sentence_length=1024
sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input: /aiffel/aiffel/transformer_chatbo/data/train_data.txt
  input_format: 
  model_prefix: /aiffel/aiffel/transformer_chatbo/data/spm_train_data
  model_type: BPE
  vocab_size: 8192
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 1024
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_outpu

bpe_model_trainer.cc(258) LOG(INFO) Added: freq=6 size=4420 all=36876 active=1829 piece=▁아픔이
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=6 size=4440 all=36863 active=1816 piece=▁의미를
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=6 size=4460 all=36858 active=1811 piece=▁지겨워
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=6 size=4480 all=36854 active=1807 piece=▁퇴근길
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=6 size=4500 all=36846 active=1799 piece=었습니다
bpe_model_trainer.cc(167) LOG(INFO) Updating active symbols. max_freq=6 min_freq=4
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=6 size=4520 all=36841 active=1832 piece=▁다양하게
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=6 size=4540 all=36831 active=1822 piece=▁선생님이
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=6 size=4560 all=36825 active=1816 piece=▁운동으로
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=6 size=4580 all=36809 active=1800 piece=▁첫사랑이
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=6 size=4600 all=36802 active

In [11]:
vocab_file = os.path.join(os.getenv('HOME'), 'aiffel/transformer_chatbo/data/spm_train_data.model')
vocab = spm.SentencePieceProcessor()
vocab.load(vocab_file)

True

In [12]:
lines = [
  data['Q+A'][0], data['Q+A'][1],data['Q+A'][2]
]
for line in lines:
    pieces = vocab.encode_as_pieces(line)
    ids = vocab.encode_as_ids(line)
    print(line)
    print(pieces)
    print(ids)

12시 땡! 하루가 또 가네요.
['▁12', '시', '▁땡', '!', '▁하루가', '▁또', '▁가네요', '.']
[5565, 7149, 3206, 7255, 4487, 210, 5936, 7108]
1지망 학교 떨어졌어 위로해 드립니다.
['▁1', '지', '망', '▁학교', '▁떨어졌어', '▁위로해', '▁드립니다', '.']
[343, 7116, 7422, 1004, 2455, 1617, 6422, 7108]
3박4일 놀러가고 싶다 여행은 언제나 좋죠.
['▁3', '박', '4', '일', '▁놀러가고', '▁싶다', '▁여행은', '▁언제나', '▁좋죠', '.']
[469, 7617, 7584, 7157, 3499, 197, 5134, 1354, 378, 7108]


In [13]:
#포지셔널 임베딩
class PositionalEmbedding(tf.keras.layers.Layer):
    def __init__(self, vocab_size, d_model):
        super(PositionalEmbedding, self).__init__()
        self.d_model = d_model
        self.embedding = tf.keras.layers.Embedding(vocab_size, d_model)
        self.pos_encoding = self.positional_encoding(10000, d_model)  

    def get_config(self):
        config = super(PositionalEmbedding, self).get_config()
        config.update({"vocab_size": self.embedding.input_dim, "d_model": self.d_model})
        return config

    def positional_encoding(self, position, d_model):
        angle_rads = self.get_angles(
            np.arange(position)[:, np.newaxis],
            np.arange(d_model)[np.newaxis, :],
            d_model
        )

        # apply sin to even indices in the array; 2i
        angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])

        # apply cos to odd indices in the array; 2i+1
        angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

        pos_encoding = angle_rads[np.newaxis, ...]

        return tf.cast(pos_encoding, dtype=tf.float32)

    def get_angles(self, pos, i, d_model):
        angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(d_model))
        return pos * angle_rates

    def call(self, x):
        length = tf.shape(x)[1]
        x = self.embedding(x)
        x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        x += self.pos_encoding[:, :length, :]
        return x


In [14]:
# 패딩 마스크, 룩어헤드 마스크 가져오기
def create_padding_mask(x):
    mask = tf.cast(tf.math.equal(x, 0), tf.float32)
  # (batch_size, 1, 1, sequence length)
    return mask[:, tf.newaxis, tf.newaxis, :]

def create_lookahead_mask(seq_len):
    mask = tf.linalg.band_part(tf.ones((seq_len, seq_len)), -1, 0)
    mask = 1 - mask
    return mask[tf.newaxis, tf.newaxis, :, :]  # (1, 1, seq_len, seq_len)

In [15]:
#스케일드 닷 프로덕트 어텐션

# 스케일드 닷 프로덕트 어텐션 함수
def scaled_dot_product_attention(query, key, value, mask):
    # 어텐션 가중치는 Q와 K의 닷 프로덕트
    matmul_qk = tf.matmul(query, key, transpose_b=True) #Q*K

    # 가중치를 정규화
    depth = tf.cast(tf.shape(key)[-1], tf.float32) #임베딩 차원 가져오기
    logits = matmul_qk / tf.math.sqrt(depth) 

    # 패딩에 마스크 추가
    if mask is not None:
        logits += (mask * -1e9)  #QK에 mask 있으면 뒤에 붙이기

    # softmax적용
    attention_weights = tf.nn.softmax(logits, axis=-1) #각 쿼리 시퀀스에 대해 모든 키 시퀀스의 중요도를 확률로 변환

    # 최종 어텐션은 가중치와 V의 닷 프로덕트
    output = tf.matmul(attention_weights, value)
    
    return output


In [16]:
# 멀티헤드 어텐션

class MultiHeadAttention(tf.keras.layers.Layer):

    def __init__(self, d_model, num_heads, name="multi_head_attention"):
        super(MultiHeadAttention, self).__init__(name=name)
        self.num_heads = num_heads
        self.d_model = d_model

        assert d_model % self.num_heads == 0

        self.depth = d_model // self.num_heads

        self.query_dense = tf.keras.layers.Dense(units=d_model)
        self.key_dense = tf.keras.layers.Dense(units=d_model)
        self.value_dense = tf.keras.layers.Dense(units=d_model)

        self.dense = tf.keras.layers.Dense(units=d_model)

    def split_heads(self, inputs, batch_size):
        inputs = tf.reshape(
            inputs, shape=(batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(inputs, perm=[0, 2, 1, 3])

    def call(self, inputs):
        query, key, value, mask = inputs['query'], inputs['key'], inputs[
            'value'], inputs['mask']
        batch_size = tf.shape(query)[0]

        # Q, K, V에 각각 Dense를 적용합니다
        query = self.query_dense(query)
        key = self.key_dense(key)
        value = self.value_dense(value)

        # 병렬 연산을 위한 머리를 여러 개 만듭니다
        query = self.split_heads(query, batch_size)
        key = self.split_heads(key, batch_size)
        value = self.split_heads(value, batch_size)

        # 스케일드 닷 프로덕트 어텐션 함수
        scaled_attention = scaled_dot_product_attention(query, key, value, mask)

        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])

        # 어텐션 연산 후에 각 결과를 다시 연결(concatenate)합니다
        concat_attention = tf.reshape(scaled_attention,
                                      (batch_size, -1, self.d_model))

        # 최종 결과에도 Dense를 한 번 더 적용합니다
        outputs = self.dense(concat_attention)

        return outputs

In [17]:
# 디코더 함수
# 이 하나의 레이어 안에는 "두 개"의 서브 레이어가 존재합니다.
def decoder_layer(units, d_model, num_heads, dropout, name="decoder_layer"):
    inputs = tf.keras.Input(shape=(None, d_model), name="inputs")
    look_ahead_mask = tf.keras.Input(shape=(1, None, None), name="look_ahead_mask")

    # 첫 번째 서브 레이어 : 멀티 헤드 어텐션 수행 (셀프 어텐션)
    attention1 = MultiHeadAttention(
      d_model, num_heads, name="attention_1")(inputs={
          'query': inputs,
          'key': inputs,
          'value': inputs,
          'mask': look_ahead_mask
      })

    # 멀티 헤드 어텐션의 결과는 LayerNormalization이라는 훈련을 돕는 테크닉을 수행
    attention1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)(attention1 + inputs)

    # 두 번째 서브레이어: FFN
    outputs = tf.keras.layers.Dense(units=units, activation='relu')(attention1)
    outputs = tf.keras.layers.Dense(units=d_model)(outputs)

    # 완전연결층의 결과는 Dropout과 LayerNormalization 수행
    outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
    outputs = tf.keras.layers.LayerNormalization(epsilon=1e-6)(outputs + attention1)

    return tf.keras.Model(
      inputs=[inputs, look_ahead_mask],outputs=outputs,name=name)

In [18]:
def decoder(num_layers,
            units,
            d_model,
            num_heads,
            dropout,
            name='decoder'):
    inputs = tf.keras.Input(shape=(None,d_model), name='inputs')
    look_ahead_mask = tf.keras.Input(shape=(1, None, None), name='look_ahead_mask')

    # Dropout이라는 훈련을 돕는 테크닉을 수행
    outputs = tf.keras.layers.Dropout(rate=dropout)(inputs)

    for i in range(num_layers):
        outputs = decoder_layer(
            units=units,
            d_model=d_model,
            num_heads=num_heads,
            dropout=dropout,
            name='decoder_layer_{}'.format(i),
        )(inputs=[outputs, look_ahead_mask])

    return tf.keras.Model(
      inputs=[inputs,look_ahead_mask],
      outputs=outputs,
      name=name)

In [19]:
#이제 GPT 함수 정의!
def gpt_model(vocab_size,num_layers,units,d_model,num_heads,dropout,name="gpt_model"):
    
    inputs = tf.keras.Input(shape=(None,), name="inputs")

    # 디코더에서 미래의 토큰 마스크,,,
    look_ahead_mask = tf.keras.layers.Lambda(
        lambda x: create_lookahead_mask(tf.shape(x)[1]),
        name='look_ahead_mask')(inputs)
    
    #임베딩 레이어
    embeddings = PositionalEmbedding(vocab_size, d_model)(inputs)

    # 디코더 호출
    decoder_model = decoder(
        num_layers=num_layers,
        units=units,
        d_model=d_model,
        num_heads=num_heads,
        dropout=dropout,
    )

    decoder_outputs = decoder_model(inputs=[embeddings, look_ahead_mask])
    
    # 완전연결층
    outputs = tf.keras.layers.Dense(units=vocab_size, name="outputs")(decoder_outputs)
    return tf.keras.Model(inputs=inputs, outputs=outputs, name=name)

In [20]:
# 모델 생성하기

tf.keras.backend.clear_session() #현재 세션 초기화하고 메모리 해제. 모델 재정의하거나 새로 생성할 때 유용

# 하이퍼파라미터

NUM_LAYERS = 6 # 디코더의 층의 개수
D_MODEL = 256 # 디코더 내부의 입, 출력의 고정 차원
NUM_HEADS = 8 # 멀티 헤드 어텐션에서의 헤드 수 
UNITS = 512 # 피드 포워드 신경망의 은닉층의 크기
DROPOUT = 0.1 # 드롭아웃의 비율
VOCAB_SIZE = 8192

model = gpt_model(
    vocab_size=VOCAB_SIZE,
    num_layers=NUM_LAYERS,
    units=UNITS,
    d_model=D_MODEL,
    num_heads=NUM_HEADS,
    dropout=DROPOUT)

model.summary()

Model: "gpt_model"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ inputs (InputLayer) │ (None, None)      │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ positional_embeddi… │ (None, None, 256) │  2,097,152 │ inputs[0][0]      │
│ (PositionalEmbeddi… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ look_ahead_mask     │ (1, 1, None,      │          0 │ inputs[0][0]      │
│ (Lambda)            │ None)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder             │ (None, None, 256) │  3,162,624 │ positional_embed… │
│ (Functional)        │                   │            │ look_ahead_mask[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ outputs (Dense)     │ (None, None,      │  2,105,344 │ decoder[0][0]     │
│                     │ 8192)             │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 7,365,120 (28.10 MB)

 Trainable params: 7,365,120 (28.10 MB)

 Non-trainable params: 0 (0.00 B)

In [21]:
# 손실함수

def loss_function(y_true, y_pred):
    y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH-1))
    y_pred = y_pred[:, :MAX_LENGTH-1, :]

    loss = tf.keras.losses.SparseCategoricalCrossentropy(
      from_logits=True, reduction='none')(y_true, y_pred)

    mask = tf.cast(tf.not_equal(y_true, 0), tf.float32)
    loss = tf.multiply(loss, mask)
    
    return tf.reduce_mean(loss)

In [22]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):

    def __init__(self, d_model, warmup_steps=4000):
        super(CustomSchedule, self).__init__()

        self.d_model = d_model
        self.d_model = tf.cast(self.d_model, tf.float32)

        self.warmup_steps = warmup_steps

    def __call__(self, step):
        step = tf.cast(step, tf.float32)
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps**-1.5)

        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

In [31]:
learning_rate = CustomSchedule(D_MODEL)

optimizer = tf.keras.optimizers.Adam(
    learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

def accuracy(y_true, y_pred):
    y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH-1))
    y_pred = tf.reshape(y_pred[:, :MAX_LENGTH - 1, :], shape=(-1, MAX_LENGTH - 1, VOCAB_SIZE))
    return tf.keras.metrics.sparse_categorical_accuracy(y_true, y_pred)

model.compile(optimizer=optimizer, loss=loss_function, metrics=[accuracy])

In [32]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

# SentencePiece 모델 로드
vocab_file = os.path.join(os.getenv('HOME'), 'aiffel/transformer_chatbo/data/spm_train_data.model')
vocab = spm.SentencePieceProcessor()
vocab.load(vocab_file)

# 최대 시퀀스 길이
MAX_LENGTH = 40

# 입력과 타깃 시퀀스를 생성하는 함수
# 입력과 타깃 시퀀스를 생성하는 함수
def create_sequences(data, vocab, max_length):
    inputs = []
    targets = []

    for sentence in data:
        tokenized_sentence = vocab.encode_as_ids(sentence)
        if len(tokenized_sentence) > max_length:
            tokenized_sentence = tokenized_sentence[:max_length]
        
        # Padding
        if len(tokenized_sentence) < max_length:
            tokenized_sentence += [vocab.pad_id()] * (max_length - len(tokenized_sentence))

        input_sequence = tokenized_sentence[:-1]
        target_sequence = tokenized_sentence[1:]+ [vocab.pad_id()]

        inputs.append(input_sequence)
        targets.append(target_sequence)

    return np.array(inputs), np.array(targets)

# 데이터를 입력 시퀀스와 타깃 시퀀스로 변환
input_sequences, target_sequences = create_sequences(all_texts, vocab, MAX_LENGTH)

# 데이터를 train/validation set으로 분리
input_train, input_val, target_train, target_val = train_test_split(
    input_sequences, target_sequences, test_size=0.2)

# 데이터셋을 텐서 형태로 변환
input_train = np.array(input_train)
target_train = np.array(target_train)
input_val = np.array(input_val)
target_val = np.array(target_val)

train_dataset = tf.data.Dataset.from_tensor_slices((input_train, target_train))
val_dataset = tf.data.Dataset.from_tensor_slices((input_val, target_val))

# 배치 크기
BATCH_SIZE = 64

# 데이터셋을 배치와 섞기 적용
train_dataset = train_dataset.cache().shuffle(len(input_train)).batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)
val_dataset = val_dataset.batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)

# 모델 훈련
model.compile(optimizer=optimizer, loss=loss_function, metrics=[accuracy])
model.fit(train_dataset, validation_data=val_dataset, epochs=20)


Epoch 1/20


InvalidArgumentError: Graph execution error:

Detected at node Reshape defined at (most recent call last):
  File "/opt/conda/lib/python3.9/runpy.py", line 197, in _run_module_as_main

  File "/opt/conda/lib/python3.9/runpy.py", line 87, in _run_code

  File "/opt/conda/lib/python3.9/site-packages/ipykernel_launcher.py", line 16, in <module>

  File "/opt/conda/lib/python3.9/site-packages/traitlets/config/application.py", line 846, in launch_instance

  File "/opt/conda/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 677, in start

  File "/opt/conda/lib/python3.9/site-packages/tornado/platform/asyncio.py", line 199, in start

  File "/opt/conda/lib/python3.9/asyncio/base_events.py", line 596, in run_forever

  File "/opt/conda/lib/python3.9/asyncio/base_events.py", line 1890, in _run_once

  File "/opt/conda/lib/python3.9/asyncio/events.py", line 80, in _run

  File "/opt/conda/lib/python3.9/site-packages/ipykernel/kernelbase.py", line 457, in dispatch_queue

  File "/opt/co